In [1]:
import pandas as pd
import numpy as np
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

SHEET_NAMES = ["Pre & Post", "Weekly", "Daily"]

In [2]:
df = pd.read_csv("data/class_data.csv", encoding = 'latin-1')
df_interested = df[df["pre_studyinterest"] == "Yes"]

In [3]:
# Verify possuble varnames by checking in bio class variables set
def find_construct_variable(df, codebook, flag):
    
    # construct possible varnames for each Name Stem in its Construct group
    # flag in ["pre", "post", 'd', 'w']
    def build_counstruct_variable_dict(codebook, flag):
        appending = [c for c in list(codebook.columns)[1:list(codebook.columns).index("Construct")]]
        if "pre" in appending:
            construct_var_dict = \
                codebook.groupby("Construct").apply(
                    lambda d: {i: [a + "_" + i for a in appending if d[d["Name Stem"] == i][a].tolist()[0] == "x"] \
                               for i in d["Name Stem"]}).to_dict()

        else:
            construct_var_dict = \
                codebook.groupby("Construct").apply(
                    lambda d: {i: [i + flag + str(a) for a in appending if d[d["Name Stem"] == i][a].tolist()[0] == "x"] \
                               for i in d["Name Stem"]}).to_dict()
        return construct_var_dict

    construct_var_dict = build_counstruct_variable_dict(codebook, flag)
    return {construct: {var_root: [var for var in construct_var_dict[construct][var_root] + [var_root] if var in df.columns] \
                          for var_root in construct_var_dict[construct]} for construct in construct_var_dict}

def print_non_exist_variable(df, to_ret):
    print("Name Stems that are not in df:  ")
    for construct in to_ret:
        for var_root in to_ret[construct]:
            if len(to_ret[construct][var_root]) == 0:
                print([construct, var_root])
    print('\n')

In [4]:
def build_var_null_percentage_df(df, sheet_name, verbal = False, overwrite = True, student_interested = False, 
                                 count_writer = None, describe_writer = None):
    
    def construct_percentage_df(construct_var_dict):
        df_percentage_master = pd.DataFrame([[construct, name_stem, var] for construct in construct_var_dict 
                                             for name_stem in construct_var_dict[construct] for var in construct_var_dict[construct][name_stem]])
        df_percentage_master.columns = ["Construct", "Name Stem", "Var"]
        df_percentage_master["count"] = df[df_percentage_master["Var"]].count().to_list()
        df_percentage_master["flag"] = [i.replace(j, '').replace('_', '') if i.replace(j, '').replace('_', '') else "original" 
                       for i, j in zip(df_percentage_master["Var"], df_percentage_master["Name Stem"])]
        df_percentage_master = pd.DataFrame(df_percentage_master.pivot(index='Name Stem', columns="flag", values=["count"]))
        return df_percentage_master
    
    def write_count_to_excel(df_percentage, df_codebook, sheet_name = sheet_name, writer = count_writer):
        appending = [c for c in list(df_codebook.columns)[1:list(df_codebook.columns).index("Construct")]]
        df_percentage_level = pd.DataFrame(df_percentage["count"]).reset_index()
        df_percentage_level = df_percentage_level.merge(df_codebook[codebook_vars], 
                                                       left_on = "Name Stem", right_on = "Name Stem", how = "left")   
        if sheet_name == "Pre & Post":
            df_percentage_level[df_percentage_level["original"].notna()].drop(["post", "pre"], axis=1).to_excel(writer, sheet_name = "Pre&Post Original")
            df_percentage_level[df_percentage_level["post"].notna()].drop(["original", "pre"], axis=1).to_excel(writer, sheet_name = "Post")
            df_percentage_level[df_percentage_level["pre"].notna()].drop(["original", "post"], axis=1).to_excel(writer, sheet_name = "Pre")
            df_percentage_level = df_percentage_level[(df_percentage_level["pre"].notna()) & (df_percentage_level["post"].notna())].drop(["original"], axis=1)
            df_percentage_level["retention_rate"] = df_percentage_level["post"] / df_percentage_level["pre"]
            df_percentage_level = df_percentage_level[["Name Stem", "pre", "post", "retention_rate", "Construct", "Item", "Response Values", "Label"]]
            df_percentage_level.to_excel(writer, sheet_name = "Pre & Post")
        else:
            df_percentage_level.to_excel(writer, sheet_name = sheet_name)    
    
    def print_percentage_head(df_percentage, df_codebook):
        df_percentage.columns = ['_'.join(col[::-1]).strip() for col in df_percentage.columns.values]
        df_percentage = pd.DataFrame(df_percentage).reset_index().merge(df_codebook[codebook_vars], 
                                                                   left_on = "Name Stem", right_on = "Name Stem", how = "left")
        df_percentage = df_percentage[codebook_vars + [c for c in df_percentage.columns if c not in codebook_vars]]
        display(df_percentage.head())
        
    def write_description_to_excel(df, construct_var_dict, sheet_name = sheet_name, writer = describe_writer, verbal = verbal):
        found_var = [var for c in construct_var_dict for var_root in construct_var_dict[c] for var in construct_var_dict[c][var_root]]
        df_describe = df[found_var].describe()
        if verbal:
            display(df_describe)
        df_describe.to_excel(writer, sheet_name = sheet_name)
    
    df_codebook =  pd.read_excel("data/codebook.xlsx", sheet_name=sheet_name)
    construct_var_dict = find_construct_variable(df, df_codebook, sheet_name[0].lower())
    df_percentage = construct_percentage_df(construct_var_dict)
    codebook_vars = ["Name Stem", "Construct", "Item", "Response Values", "Label"]
    
    if verbal:
        print(sheet_name, '\n\n')
        print_non_exist_variable(df, construct_var_dict)
        print_percentage_head(df_percentage.copy(), df_codebook)
    
    if overwrite:
        write_count_to_excel(df_percentage.copy(), df_codebook)
        write_description_to_excel(df, construct_var_dict)
  
    return df_percentage
        

## Interested Students

In [5]:
fname_count = "data/student_interested_var_count.xlsx"
fname_describe = "data/student_interested_var_describe.xlsx"
print("Total number of students:  ", df_interested.shape[0])
with pd.ExcelWriter(fname_count) as count_writer, pd.ExcelWriter(fname_describe) as describe_writer:
    for sheet_name in SHEET_NAMES:
        build_var_null_percentage_df(df_interested, sheet_name, verbal = False, student_interested=True, 
                                     count_writer = count_writer, describe_writer = describe_writer)

Total number of students:   99


## All Students

In [6]:
# fname = "data/student_var_count.xlsx"
# fname_describe = "data/student_var_describe.xlsx"
# print("Total number of students:  ", df_interested.shape[0])
# with pd.ExcelWriter(fname) as count_writer, pd.ExcelWriter(fname_describe) as describe_writer:
#     for sheet_name in SHEET_NAMES:
#         build_var_null_percentage_df(df, sheet_name, verbal = False, student_interested=False, 
#                                      count_writer = count_writer, describe_writer = describe_writer)

## Select Var based on count percentage

In [11]:
def build_decision_df(df, sheet_name, threshold = 0.5, writer = None):
    df_codebook =  pd.read_excel("data/codebook.xlsx", sheet_name=sheet_name)
    construct_var_dict = find_construct_variable(df, df_codebook, sheet_name[0].lower())
    df_master = pd.DataFrame([[construct, name_stem, var] for construct in construct_var_dict 
                              for name_stem in construct_var_dict[construct] for var in construct_var_dict[construct][name_stem]])
    df_master.columns = ["Construct", "Name Stem", "Var"]
    df_master["Percentage"] = df[df_master["Var"]].count().to_list()
    df_master["Percentage"] /= df.shape[0]
    df_master["Decision"] = ["Keep" if i >= threshold else "Drop" for i in df_master["Percentage"]]
    codebook_vars = ["Name Stem", "Item", "Label", "Response Values"]
    df_master = df_master.merge(df_codebook[codebook_vars],left_on = "Name Stem", right_on = "Name Stem", how = "left") 
    
    
    if writer:
        df_master.to_excel(writer, sheet_name = sheet_name)
        for dec in ["Keep", "Drop"]:
            df_dec = df_master[df_master["Decision"] == dec]
            df_dec.reset_index(drop=True).to_excel(writer, sheet_name = "{} {}".format(sheet_name, dec))
        df_temp = pd.DataFrame(df_master.groupby("Construct").mean().reset_index())
        df_temp["Decision"] = ["Keep" if i >= threshold else "Drop" for i in df_temp["Percentage"]]
        df_temp.to_excel(writer, sheet_name = "{} AGG Decision".format(sheet_name))

In [12]:
fname = "data/student_interested_var_decision.xlsx"
with pd.ExcelWriter(fname) as decision_writer:
    for sheet_name in SHEET_NAMES:
        build_decision_df(df_interested, sheet_name, writer = decision_writer)